In [1]:
import sys
print(sys.executable)

/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/bin/python


In [2]:
import torch
import sys
import os

print("Python:", sys.executable)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")
print("Realpath:", os.path.realpath(sys.executable))
print(torch.__file__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())

device = torch.device("cuda:1")





Python: /workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/bin/python
CUDA available: True
GPU name: NVIDIA RTX A6000
Realpath: /opt/conda/bin/python3.11
/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/lib/python3.11/site-packages/torch/__init__.py
12.4
90100


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
import sys
print(sys.executable)

import os
os.environ["TRANSFORMERS_NO_TF"] = "1"




/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/bin/python


In [5]:
import os

import pprint
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings


from langchain_core.documents import Document


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv(r"C:\Users\Rhm_EBR\TTPFShot_mitsu\csvfiles\TPPHunter_T_Again_FINAL.csv")

In [7]:
!nvidia-smi

Thu Dec 18 16:46:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:03:00.0 Off |                  Off |
| 30%   38C    P8             25W /  300W |      28MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
df

,ID,Domain,act_Technique ID,act_TacticsName_before,act_Technique Name,Sentences,test_or_train,Predicted_ATT&CK ID_original,Predicted_TacticsName_before,act_TacticsName,Predicted_TacticsName
0,1,Enterprise,T1543,Persistence:Privilege Escalation,Create or Modify System Process: Windows Service,Carbanak malware installs itself as a service ...,train,NaN,NaN,Persistence:Privilege Escalation,NaN
1,2,Enterprise,T1562,Defense Evasion,Impair Defenses: Disable or Modify System Fire...,Carbanak may use netsh to add local firewall r...,train,NaN,NaN,Defense Evasion,NaN
2,3,Enterprise,T1036,Defense Evasion,Masquerading: Masquerade Task or Service,Carbanak has copied legitimate service names t...,train,NaN,NaN,Defense Evasion,NaN
3,4,Enterprise,T1036,Defense Evasion,Masquerading: Match Legitimate Name or Location,"Carbanak has named malware ""svchost.exe,"" whic...",train,NaN,NaN,Defense Evasion,NaN
4,5,Enterprise,T1588,Resource Development,Obtain Capabilities: Tool,Carbanak has obtained and used open-source too...,train,NaN,NaN,Resource Development,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8382,8383,Enterprise,T1573,Command and Control,Encrypted Channel: Symmetric Cryptography,NETEAGLE will decrypt resources it downloads w...,train,NaN,NaN,Command and Control,NaN
8383,8384,Enterprise,T1041,Exfiltration,Exfiltration Over C2 Channel,NETEAGLE is capable of reading files over the ...,train,NaN,NaN,Exfiltration,NaN
8384,8385,Enterprise,T1083,Discovery,File and Directory Discovery,NETEAGLE allows adversaries to enumerate and m...,train,NaN,NaN,Discovery,NaN
8385,8386,Enterprise,T1057,Discovery,Process Discovery,NETEAGLE can send process listings over the C2...,test,T1057,Exfiltration,Discovery,Command and Control


In [9]:
len(set(df["act_Technique ID"]))
print(df.columns.tolist())

['ID', 'Domain', 'act_Technique ID', 'act_TacticsName_before', 'act_Technique Name', 'Sentences', 'test_or_train', 'Predicted_ATT&CK ID_original', 'Predicted_TacticsName_before', 'act_TacticsName', 'Predicted_TacticsName']


In [10]:
df = df[["ID", "test_or_train", "act_Technique ID", "act_Technique Name","act_TacticsName", "Sentences", "Predicted_TacticsName"]]

In [11]:
# train, test = train_test_split(df, test_size=0.2, random_state=0)
import pandas as pd
import numpy as np
from pathlib import Path


# test_or_train が "train" の行だけ
train = df[df["test_or_train"] == "train"].copy()

# test_or_train が "test" の行だけ
# txtからIDリスト読み込み（/mnt/data にある）
id_path = Path("/workspace/miluser/Mitsuharu_2002/csv_data/collectIDs.txt")
keep_ids = sorted({int(x) for x in id_path.read_text().split()})
test = df[df["test_or_train"] == "test"].copy()
# IDが数値/文字列どっちでも動くように一旦数値化してフィルタ
test = test[pd.to_numeric(test["ID"], errors="coerce").isin(keep_ids)].copy()

In [12]:
print(len(test))
print(len(train))

198
6709


In [13]:
train

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
0,1,train,T1543,Create or Modify System Process: Windows Service,Persistence:Privilege Escalation,Carbanak malware installs itself as a service ...,NaN
1,2,train,T1562,Impair Defenses: Disable or Modify System Fire...,Defense Evasion,Carbanak may use netsh to add local firewall r...,NaN
2,3,train,T1036,Masquerading: Masquerade Task or Service,Defense Evasion,Carbanak has copied legitimate service names t...,NaN
3,4,train,T1036,Masquerading: Match Legitimate Name or Location,Defense Evasion,"Carbanak has named malware ""svchost.exe,"" whic...",NaN
4,5,train,T1588,Obtain Capabilities: Tool,Resource Development,Carbanak has obtained and used open-source too...,NaN
...,...,...,...,...,...,...,...
8379,8380,train,T1071,Web Protocols,Command and Control,NETEAGLE will attempt to detect if the infecte...,NaN
8380,8381,train,T1547,Boot or Logon Autostart Execution: Registry Ru...,Persistence:Privilege Escalation,"The ""SCOUT"" variant of NETEAGLE achieves persi...",NaN
8382,8383,train,T1573,Encrypted Channel: Symmetric Cryptography,Command and Control,NETEAGLE will decrypt resources it downloads w...,NaN
8383,8384,train,T1041,Exfiltration Over C2 Channel,Exfiltration,NETEAGLE is capable of reading files over the ...,NaN


In [14]:
test

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
22,23,test,T1566,Phishing: Spearphishing Attachment,Initial Access,Frankenstein has used spearphishing emails to ...,Collection
31,32,test,T1047,Windows Management Instrumentation,Execution,Frankenstein has used WMI queries to check if ...,Discovery
32,33,test,T1087,Account Discovery: Local Account,Discovery,Ke3chang performs account discovery using comm...,Discovery
48,49,test,T1036,Masquerading: Right-to-Left Override,Defense Evasion,Ke3chang has used the right-to-left override c...,Defense Evasion
49,50,test,T1036,Masquerading: Match Legitimate Name or Location,Defense Evasion,Ke3chang has dropped their malware into legiti...,Defense Evasion
...,...,...,...,...,...,...,...
8168,8169,test,T1055,Process Injection: Dynamic-link Library Injection,Defense Evasion:Privilege Escalation,Aria-body has the ability to inject itself int...,Defense Evasion
8255,8256,test,T1082,System Information Discovery,Discovery,Pupy can grab a system’s information including...,Discovery
8321,8322,test,T1059,Command and Scripting Interpreter: Windows Com...,Execution,Cobian RAT can launch a remote command shell i...,Command and Control
8355,8356,test,T1053,Scheduled Task/Job: Scheduled Task,Execution:Persistence:Privilege Escalation,JSS Loader has the ability to launch scheduled...,Persistence


In [15]:
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
model = SentenceTransformer("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

docs = []
texts = []
embeddings_list = []

for index, row in train.iterrows():
    text = row["Sentences"]
    emb = model.encode(text, convert_to_numpy=True)

    doc = Document(
        page_content=text,
        metadata={
            "TTP_ID": row["act_Technique ID"],        # ATT&CK ID（T1059とか）
            "TechniqueName": row["act_Technique Name"],  # 人間向けの名前
            "Tactics": row["act_TacticsName"],       # 大項目 (Execution;Persistence など)
        },
    )

    docs.append(doc)
    texts.append(text)
    embeddings_list.append(emb)



/tmp/ipykernel_1462250/2600432364.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")


In [16]:
docs[0]

Document(metadata={'TTP_ID': 'T1543', 'TechniqueName': 'Create or Modify System Process: Windows Service', 'Tactics': 'Persistence:Privilege Escalation'}, page_content='Carbanak malware installs itself as a service to provide persistence and SYSTEM privileges.[1]')

In [17]:
import faiss
import numpy as np

# ① ベクトル次元
d = embeddings_list[0].shape[0]  # 384 次元とか

# ② FAISS index を作成（L2距離）
index = faiss.IndexFlatL2(d)

# ③ numpy 配列にまとめる
emb_matrix = np.vstack(embeddings_list).astype('float32')

# ④ インデックスへ追加
index.add(emb_matrix)

print("FAISS に追加された件数:", index.ntotal)



FAISS に追加された件数: 6709


In [18]:
# 類似検索テスト
query = model.encode(test.iloc[0]["Sentences"], convert_to_numpy=True).astype('float32')

k = 45
distances, indices = index.search(query.reshape(1, -1), k)

print(indices)
print(distances)


[[ 823 1249   62 1163 2337  364  461 1267 1644  564  499 1085  496  262
   986  984 1002 6678 4087 1066  458 1645 3768  238  477 4091 1662 1107
   167 3726  417 1266  264  327 1701  825  267  989 1050  444   91  270
   113 1086  387]]
[[0.48680943 0.48761934 0.49586755 0.5595252  0.5632787  0.57167006
  0.58027506 0.61202383 0.61607087 0.6236972  0.6646012  0.7110164
  0.7215114  0.7336979  0.7465569  0.7497904  0.76443976 0.7659313
  0.7688658  0.77171636 0.77479446 0.77480364 0.77520066 0.778413
  0.7956972  0.79633945 0.7970606  0.7993661  0.7994639  0.80153227
  0.8080427  0.8176934  0.8215108  0.83157974 0.8321495  0.83506817
  0.83898425 0.8401412  0.8435478  0.84549224 0.84577346 0.8517759
  0.85512817 0.85515213 0.8595083 ]]


In [19]:
K = 45
recall_hit = 0

for idx, row in test.iterrows():
    query_vec = model.encode(row["Sentences"], convert_to_numpy=True).astype('float32')
    distances, indices = index.search(query_vec.reshape(1, -1), K)

    # 候補に対応する TTP を取り出す
    retrieved_ttps = [docs[i].metadata["TTP_ID"] for i in indices[0]]

    if row["act_Technique ID"] in retrieved_ttps:
        recall_hit += 1

print("Recall@{} = {:.4f}".format(K, recall_hit / len(test)))




Recall@45 = 0.9798


In [20]:
import pandas as pd
import numpy as np
from pathlib import Path

K = 45

# どの列が入力文か（あなたのコードに合わせて）
TEXT_COL = "Sentences"          # ここはそのまま
GOLD_COL = "act_Technique ID"   # goldの列名が違うなら変更

# ID列が無い場合は、indexからIDを作る（※あなたはIDズレがあったのでここ重要）
if "ID" not in test.columns:
    test = test.copy()
    test["ID"] = test.index.astype(int) + 1  # 1始まりに寄せる（必要なら +0 に）

# ===== 1) クエリをまとめて埋め込み（速い） =====
queries = test[TEXT_COL].astype(str).tolist()
qvecs = model.encode(queries, convert_to_numpy=True).astype("float32")

# ===== 2) FAISSでまとめて検索 =====
distances, indices = index.search(qvecs, K)   # (N,K)

# ===== 3) 候補を縦持ちにして保存 =====
rows = []
for qi in range(len(test)):
    q_id = int(test.iloc[qi]["ID"])
    q_text = test.iloc[qi][TEXT_COL]
    gold = str(test.iloc[qi][GOLD_COL]) if GOLD_COL in test.columns else ""

    # goldが候補に入ってるか（rankも取る）
    retrieved_ids = [str(docs[j].metadata.get("TTP_ID", "")) for j in indices[qi]]
    try:
        hit_rank = retrieved_ids.index(str(gold)) + 1
    except ValueError:
        hit_rank = 0

    for rank, (d, j) in enumerate(zip(distances[qi], indices[qi]), start=1):
        md = getattr(docs[j], "metadata", {}) or {}
        cand_id = str(md.get("TTP_ID", ""))
        cand_tac = str(md.get("TacticsName", ""))  # あれば
        cand_name = str(md.get("TechniqueName", ""))  # あれば

        # cand_text（docs側に本文がある場合）
        cand_text = ""
        if hasattr(docs[j], "page_content"):
            cand_text = docs[j].page_content
        elif "text" in md:
            cand_text = str(md.get("text", ""))

        rows.append({
            "ID": q_id,
            "rank": rank,
            "score": float(d),            # FAISSの生スコア（L2なら距離、IPなら内積）
            "cand_TTP_ID": cand_id,
            "cand_TacticsName": cand_tac,
            "cand_TechniqueName": cand_name,
            "cand_text": cand_text,
            "gold_id": gold,
            "gold_in_topK": (hit_rank > 0),
            "gold_rank": hit_rank,
            "query_text": q_text,
        })

out_df = pd.DataFrame(rows)

# 保存
out_dir = Path("./retrieval_dump")
out_dir.mkdir(parents=True, exist_ok=True)

out_parquet = out_dir / f"retrieval_{K}_per_id.parquet"
out_csv = out_dir / f"retrieval_{K}_per_id.csv"

out_df.to_parquet(out_parquet, index=False)
out_df.to_csv(out_csv, index=False, encoding="utf-8-sig")

print("Saved:", out_parquet)
print("Saved:", out_csv)
print("rows:", out_df.shape)
print("unique IDs:", out_df["ID"].nunique())
print("gold_in_topK rate:", out_df.drop_duplicates("ID")["gold_in_topK"].mean())


Saved: retrieval_dump/retrieval_45_per_id.parquet
Saved: retrieval_dump/retrieval_45_per_id.csv
rows: (8910, 11)
unique IDs: 198
gold_in_topK rate: 0.9797979797979798


In [35]:
df

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
0,1,train,T1543,Create or Modify System Process: Windows Service,Persistence:Privilege Escalation,Carbanak malware installs itself as a service ...,NaN
1,2,train,T1562,Impair Defenses: Disable or Modify System Fire...,Defense Evasion,Carbanak may use netsh to add local firewall r...,NaN
2,3,train,T1036,Masquerading: Masquerade Task or Service,Defense Evasion,Carbanak has copied legitimate service names t...,NaN
3,4,train,T1036,Masquerading: Match Legitimate Name or Location,Defense Evasion,"Carbanak has named malware ""svchost.exe,"" whic...",NaN
4,5,train,T1588,Obtain Capabilities: Tool,Resource Development,Carbanak has obtained and used open-source too...,NaN
...,...,...,...,...,...,...,...
8382,8383,train,T1573,Encrypted Channel: Symmetric Cryptography,Command and Control,NETEAGLE will decrypt resources it downloads w...,NaN
8383,8384,train,T1041,Exfiltration Over C2 Channel,Exfiltration,NETEAGLE is capable of reading files over the ...,NaN
8384,8385,train,T1083,File and Directory Discovery,Discovery,NETEAGLE allows adversaries to enumerate and m...,NaN
8385,8386,test,T1057,Process Discovery,Discovery,NETEAGLE can send process listings over the C2...,Command and Control


# 生成テスト用（TacticsのBERT)

In [36]:
from collections import Counter
import numpy as np

K = 45  # さっきと同じK

# ① number件目のテストデータを取り出す
number = 3
a = number - 1
row = test.iloc[a]

query_text = row["Sentences"]
true_tid   = row["act_Technique ID"]
true_tac   = row["act_TacticsName"]   # 正解Tactics

# ② ベクトル化して近傍検索
query_vec = model.encode(query_text, convert_to_numpy=True).astype("float32")
distances, indices = index.search(query_vec.reshape(1, -1), K)

# ③ 上位K件の Tactics を取り出す
top_k_indices = indices[0]
top_k_dists   = distances[0]

retrieved_tactics = [docs[i].metadata["Tactics"] for i in top_k_indices]

# もし "Tactics" が "A;B" みたいな形の可能性があるなら分割した版も作れる
# split_tactics = [[t.strip() for t in str(docs[i].metadata["Tactics"]).split(";") if t.strip()]
#                  for i in top_k_indices]

# ④ 出力
print(f"===== {number}件目のチェック =====")
print(f"Technique ID (真値) : {true_tid}")
print(f"Tactics (真値)      : {true_tac}")
print()
print(f"入力CTI文:\n{query_text}\n")
print(f"上位 {K} 件の候補 Tactics:")

for rank, (idx_i, dist, tac) in enumerate(zip(top_k_indices, top_k_dists, retrieved_tactics), start=1):
    print(f"  {rank}. Tactics={tac}  (doc_index={idx_i}, dist={dist:.4f})")

print()
print(f"真の Tactics が候補に含まれているか？ -> {true_tac in retrieved_tactics}")


===== 3件目のチェック =====
Technique ID (真値) : T1087
Tactics (真値)      : Discovery

入力CTI文:
Ke3chang performs account discovery using commands such as net localgroup administrators and net group "REDACTED" /domain on specific permissions groups.[1]

上位 45 件の候補 Tactics:
  1. Tactics=Discovery  (doc_index=40, dist=0.3576)
  2. Tactics=Credential Access  (doc_index=36, dist=0.8145)
  3. Tactics=Defense Evasion:Persistence:Privilege Escalation:Initial Access  (doc_index=49, dist=0.8297)
  4. Tactics=Discovery  (doc_index=45, dist=0.9313)
  5. Tactics=Discovery  (doc_index=41, dist=0.9423)
  6. Tactics=Credential Access  (doc_index=37, dist=0.9492)
  7. Tactics=Credential Access  (doc_index=39, dist=0.9492)
  8. Tactics=Credential Access  (doc_index=38, dist=0.9616)
  9. Tactics=Lateral Movement  (doc_index=42, dist=0.9695)
  10. Tactics=Discovery  (doc_index=33, dist=0.9787)
  11. Tactics=Discovery  (doc_index=1946, dist=0.9809)
  12. Tactics=Discovery  (doc_index=2605, dist=1.0129)
  13. Tactic

In [37]:
from langchain_community.vectorstores import FAISS as LCFAISS
db = LCFAISS.from_documents(docs, embeddings)
db.save_local("TTPHunter_50_TTPs_andTactics_5")

In [38]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS as LCFAISS

# 1. 保存時と同じモデル名で embeddings（モデル）を復元
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
    # SecureBERT 使って保存したなら、ここも同じ名前にする
)

# 2. ローカルからベクターストアをロード
db = LCFAISS.load_local(
    "TTPHunter_50_TTPs_andTactics_5",
    embeddings,
    allow_dangerous_deserialization=True
)

print("loaded FAISS index:", db)

docs = db.similarity_search("test", k=1)
doc = docs[0]
print(doc.metadata)  # ここに "Tactics" が見えてたらOK


loaded FAISS index: <langchain_community.vectorstores.faiss.FAISS object at 0x7f7ec3c98110>
{'TTP_ID': 'T1588', 'TechniqueName': 'Obtain Capabilities: Tool', 'Tactics': 'Resource Development'}


In [39]:
test

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
22,23,test,T1566,Phishing: Spearphishing Attachment,Initial Access,Frankenstein has used spearphishing emails to ...,Collection
31,32,test,T1047,Windows Management Instrumentation,Execution,Frankenstein has used WMI queries to check if ...,Discovery
32,33,test,T1087,Account Discovery: Local Account,Discovery,Ke3chang performs account discovery using comm...,Discovery
48,49,test,T1036,Masquerading: Right-to-Left Override,Defense Evasion,Ke3chang has used the right-to-left override c...,Defense Evasion
49,50,test,T1036,Masquerading: Match Legitimate Name or Location,Defense Evasion,Ke3chang has dropped their malware into legiti...,Defense Evasion
...,...,...,...,...,...,...,...
8168,8169,test,T1055,Process Injection: Dynamic-link Library Injection,Defense Evasion:Privilege Escalation,Aria-body has the ability to inject itself int...,Defense Evasion
8255,8256,test,T1082,System Information Discovery,Discovery,Pupy can grab a system’s information including...,Discovery
8321,8322,test,T1059,Command and Scripting Interpreter: Windows Com...,Execution,Cobian RAT can launch a remote command shell i...,Command and Control
8355,8356,test,T1053,Scheduled Task/Job: Scheduled Task,Execution:Persistence:Privilege Escalation,JSS Loader has the ability to launch scheduled...,Persistence


### Few-Shot Learning

In [40]:
import sys
print(sys.executable)


/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/bin/python


In [41]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/workspace/miluser/Mitsuharu_2002


# llama 読み込み

In [42]:
# !pip install llama-cpp-python

from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="unsloth/Llama-3.3-70B-Instruct-GGUF",
	filename="Llama-3.3-70B-Instruct-Q5_K_M.gguf",
	cache_dir = "/workspace/miluser/Mitsuharu_2002/llama_cache",
    n_ctx=4000,
    n_gpu_layers=-1,          # できるだけ GPU に載せる
    main_gpu=0,               # 小さいテンソルは GPU0 で処理
    tensor_split=[0.5, 0.5],  # 大きいテンソルを GPU0/1 に 50:50 で分割
    verbose=False,             # ロードログを出す
)

llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is the capital of France?"
		}
	]
)


llama_context: n_ctx_per_seq (4000) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


{'id': 'chatcmpl-c2343975-91f9-46c4-8e0c-0f1603ac066b',
 'object': 'chat.completion',
 'created': 1766036744,
 'model': '/workspace/miluser/Mitsuharu_2002/llama_cache/models--unsloth--Llama-3.3-70B-Instruct-GGUF/snapshots/8f14c5c5d06fca109ff16f94496147a8479711de/./Llama-3.3-70B-Instruct-Q5_K_M.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'The capital of France is Paris.'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 42, 'completion_tokens': 7, 'total_tokens': 49}}

In [43]:
df

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
0,1,train,T1543,Create or Modify System Process: Windows Service,Persistence:Privilege Escalation,Carbanak malware installs itself as a service ...,NaN
1,2,train,T1562,Impair Defenses: Disable or Modify System Fire...,Defense Evasion,Carbanak may use netsh to add local firewall r...,NaN
2,3,train,T1036,Masquerading: Masquerade Task or Service,Defense Evasion,Carbanak has copied legitimate service names t...,NaN
3,4,train,T1036,Masquerading: Match Legitimate Name or Location,Defense Evasion,"Carbanak has named malware ""svchost.exe,"" whic...",NaN
4,5,train,T1588,Obtain Capabilities: Tool,Resource Development,Carbanak has obtained and used open-source too...,NaN
...,...,...,...,...,...,...,...
8382,8383,train,T1573,Encrypted Channel: Symmetric Cryptography,Command and Control,NETEAGLE will decrypt resources it downloads w...,NaN
8383,8384,train,T1041,Exfiltration Over C2 Channel,Exfiltration,NETEAGLE is capable of reading files over the ...,NaN
8384,8385,train,T1083,File and Directory Discovery,Discovery,NETEAGLE allows adversaries to enumerate and m...,NaN
8385,8386,test,T1057,Process Discovery,Discovery,NETEAGLE can send process listings over the C2...,Command and Control


In [44]:
import re
import pandas as pd
from llama_cpp import Llama

import pyarrow as pa
import pyarrow.parquet as pq

# ---------------------------
# 便利関数
# ---------------------------
def norm_id(x: str) -> str:
    return str(x).strip()

def split_tacs_anysep(x):
    """
    Tactics文字列を分割して list[str] にする。
    gold が ":" 区切りのことがある / pred が ";" 区切りのことがある -> [;:] で split
    """
    if x is None:
        return []
    # pandasのNA判定は str でも飛んでくるので安全に
    try:
        if pd.isna(x):
            return []
    except Exception:
        pass

    s = str(x).strip()
    if not s or s.lower() == "nan":
        return []
    parts = re.split(r"[;:]", s)
    return [p.strip() for p in parts if p.strip()]

def any_overlap(a, b) -> bool:
    return len(set(a) & set(b)) > 0

# ---------------------------
# 事前準備
# ---------------------------

# 正規の Technique ID → Tactics / Name 対応表
ttp_map = pd.read_csv("/workspace/miluser/Mitsuharu_2002/csv_data/TPPHunter_T_Again_FINAL.csv")

# 余計な空白・NaN対策
ttp_map["act_Technique ID"]   = ttp_map["act_Technique ID"].astype(str).str.strip()
ttp_map["act_TacticsName"]    = ttp_map["act_TacticsName"].astype(str).str.strip()
ttp_map["act_Technique Name"] = ttp_map["act_Technique Name"].astype(str)

# ★重要：Technique→Tactics は dict(zip) で潰さない（複数Tactics保持）
id2tac = (
    ttp_map.groupby("act_Technique ID")["act_TacticsName"]
    .apply(lambda s: ";".join(sorted(set(t for x in s for t in split_tacs_anysep(x)))))
    .to_dict()
)

# Name は最初の1個でOK（同一IDで同名のはず）
id2name = (
    ttp_map.groupby("act_Technique ID")["act_Technique Name"]
    .first()
    .to_dict()
)

def build_prompt(text_to_predict: str, candidate_ids_ranked, examples_block: str, pred_tacs):
    cand_lines = []
    for cid in candidate_ids_ranked:
        tech_name = id2name.get(cid, "")
        tac_name  = id2tac.get(cid, "")
        cand_lines.append(f"- {cid}: {tech_name} (Tactics: {tac_name})")
    cand_block = "\n".join(cand_lines)

    pred_hint = ";".join(pred_tacs) if pred_tacs else "(unknown)"

    prompt = (
        "You are a cybersecurity expert.\n"
        "Your task is to classify the last sentence of 'InputText' into exactly ONE ATT&CK Technique ID.\n"
        "You are given example sentences with similarity scores and their Technique IDs.\n"
        "Smaller similarity score means more similar.\n\n"
        f"Predicted Tactics (may be noisy): {pred_hint}\n"
        "Prefer Technique IDs whose Tactics include the Predicted Tactics, but if none fit well, choose the best overall match.\n\n"
        "Allowed Technique IDs:\n"
        f"{cand_block}\n\n"
        "Rules:\n"
        "- You MUST answer with exactly ONE Technique ID.\n"
        "- The answer MUST be one of the Allowed Technique IDs above.\n"
        "- The ID MUST match: T#### or T####.###\n"
        "- Do NOT output explanations.\n\n"
        "Examples:\n"
        f"{examples_block}\n"
        f"InputText:{text_to_predict.lower().strip()}\n"
        "Technique ID:"
    )
    return prompt

def parse_llama_output(out, candidate_ids_ranked):
    raw = out["choices"][0]["message"]["content"].strip()
    first_line = raw.splitlines()[0].strip()

    m = re.search(r"T\d{4}(?:\.\d{3})?", first_line)
    if m:
        cid = m.group(0)
        if cid in candidate_ids_ranked:
            return cid, raw

    for cid in candidate_ids_ranked:
        if cid in first_line:
            return cid, raw

    # ★最終fallbackは「スコア順先頭」
    return (candidate_ids_ranked[0] if candidate_ids_ranked else ""), raw

# ---------------------------
# 本番ループ（ATT&CK ID 予測）
# ---------------------------

predicted_labels = []
logs = []

# ★追加：top55候補保存用
retrieval_rows = []

K_NEIGHBORS = 65          # 近傍数
MAX_CAND_IDS = 45         # LLMに渡すAllowed数（長すぎると崩れる）
MIN_TACTICS_CANDS = 3     # tactics一致が少なすぎる時の“拡張”ライン
MAX_EXAMPLES = 65         # few-shot例の最大数（重いなら減らしてOK）
SAVE_TOPK = 55            # ★保存したい候補数

for index, row in test.iterrows():
    text_to_predict = row["Sentences"]

    actual_id    = norm_id(row["act_Technique ID"])
    gold_tac_raw = row["act_TacticsName"]                 # ":" を含む可能性
    pred_tac_raw = row.get("Predicted_TacticsName", None) # ";" の可能性

    gold_tacs = split_tacs_anysep(gold_tac_raw)
    pred_tacs = split_tacs_anysep(pred_tac_raw)

    # 1) 近傍取得
    search_results = db.similarity_search_with_score(text_to_predict, k=K_NEIGHBORS)

    # 2) score < 2 のフィルタ
    base_candidates = [(doc, score) for (doc, score) in search_results if score < 2]

    if not base_candidates:
        predicted_labels.append("")
        print(f"[{index}] no base candidates | gold_id={actual_id} gold_tac={gold_tac_raw}")
        continue

    # 3) 同じTechniqueが何回も出るので「IDごとに最小scoreの1件」に圧縮
    best_by_id = {}
    for doc, score in base_candidates:
        cid = norm_id(doc.metadata.get("TTP_ID", ""))
        if not cid:
            continue
        if cid not in best_by_id or score < best_by_id[cid][0]:
            best_by_id[cid] = (score, doc)

    # baseをスコア順に並べた「IDランキング」
    base_ranked = sorted(best_by_id.items(), key=lambda x: x[1][0])  # [(cid, (score, doc)), ...]

    # 4) Predicted Tactics でフィルタ（ただし絞りすぎは拡張）
    tactics_ranked = []
    for cid, (score, doc) in base_ranked:
        tac_str = id2tac.get(cid, doc.metadata.get("Tactics", ""))
        cand_tacs = split_tacs_anysep(tac_str)

        if (not pred_tacs) or any_overlap(pred_tacs, cand_tacs):
            tactics_ranked.append((cid, score, doc, cand_tacs))

    used_fallback = False
    if len(tactics_ranked) >= MIN_TACTICS_CANDS:
        ranked_final = tactics_ranked
    else:
        used_fallback = True
        ranked_final = tactics_ranked[:]
        used_ids = set(cid for cid, *_ in ranked_final)

        for cid, (score, doc) in base_ranked:
            if cid in used_ids:
                continue
            tac_str = id2tac.get(cid, doc.metadata.get("Tactics", ""))
            cand_tacs = split_tacs_anysep(tac_str)
            ranked_final.append((cid, score, doc, cand_tacs))
            used_ids.add(cid)
            if len(ranked_final) >= MIN_TACTICS_CANDS:
                break

    ranked_final = ranked_final[:MAX_CAND_IDS]
    candidate_ids_ranked = [cid for cid, *_ in ranked_final]

    if not candidate_ids_ranked:
        predicted_labels.append("")
        print(f"[{index}] no candidates after filter | gold_id={actual_id} gold_tac={gold_tac_raw}")
        continue

    # ★ ここで top55 を保存（ID圧縮後の base_ranked を使う）
    passed_ids = set(cid for cid, *_ in tactics_ranked)  # tacticsフィルタを通ったID（拡張前の素フィルタ）
    top55 = base_ranked[:SAVE_TOPK]

    for r, (cid, (score, doc)) in enumerate(top55, start=1):
        retrieval_rows.append({
            "index": int(index),  # testの行番号
            "rank": int(r),
            "score": float(score),
            "cand_TTP_ID": cid,
            "cand_TacticsName": id2tac.get(cid, ""),
            "cand_TechniqueName": id2name.get(cid, ""),
            "cand_text": doc.page_content,
            "passed_tactics_filter": int(cid in passed_ids),
            "used_fallback": int(used_fallback),
            "gold_id": actual_id,
            "gold_tac_raw": "" if gold_tac_raw is None else str(gold_tac_raw),
            "predicted_tactics_raw": "" if pred_tac_raw is None else str(pred_tac_raw),
        })

    # 5) Examplesブロック（LLM入力用）
    xs = []
    for cid, score, doc, _cand_tacs in ranked_final[:MAX_EXAMPLES]:
        xs.append(
            "\nText:" + doc.page_content.lower().strip() +
            "\nSimilarity:" + str(score) + "\n" +
            "Technique ID:" + cid + "\n"
        )
    examples_block = "".join(xs)

    # 6) Llama推論
    prompt = build_prompt(text_to_predict, candidate_ids_ranked, examples_block, pred_tacs)

    out = llm.create_chat_completion(
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=32,
    )

    pred_id, raw_out = parse_llama_output(out, candidate_ids_ranked)
    predicted_labels.append(pred_id)

    # --------------- ログ用チェック ---------------
    pred_id_tacs = split_tacs_anysep(id2tac.get(pred_id, ""))

    PredTacMatch = "OK" if (pred_tacs and any_overlap(pred_tacs, pred_id_tacs)) else "NO"
    GoldTacMatch = "OK" if (gold_tacs and any_overlap(gold_tacs, pred_id_tacs)) else "NO"

    if pred_tacs and (not any_overlap(pred_tacs, pred_id_tacs)) and (pred_id == actual_id):
        check = "PredTacticsMismatchButCorrect"
    else:
        check = "OK"

    print(
        f"[{index}] fallback={used_fallback} | "
        f"gold_id={actual_id} gold_tac={gold_tac_raw} | "
        f"pred_id={pred_id} pred_id_tac={'/'.join(pred_id_tacs)} | "
        f"Predicted(Tactics)={pred_tac_raw} | "
        f"PredTacMatch={PredTacMatch} | "
        f"GoldTacMatch={GoldTacMatch} | "
        f"Check={check}"
    )

    logs.append({
        "index": int(index),
        "fallback": used_fallback,
        "gold_id": actual_id,
        "gold_tac_raw": "" if gold_tac_raw is None else str(gold_tac_raw),
        "pred_id": pred_id,
        "pred_id_tactics": ";".join(pred_id_tacs),
        "predicted_tactics_raw": "" if pred_tac_raw is None else str(pred_tac_raw),
        "PredTacMatch": PredTacMatch,
        "GoldTacMatch": GoldTacMatch,
        "check": check,
        "llm_raw": raw_out,
    })

# ---------------------------
# ループ後：top55候補を Parquet に保存（pyarrow直）
# ---------------------------
retrieval_df = pd.DataFrame(retrieval_rows)

table = pa.Table.from_pandas(retrieval_df, preserve_index=False)
pq.write_table(table, "retrieval_55_per_id_tactics_constraint.parquet")

# ついでにログも保存したいなら（軽いCSVでOK）
logs_df = pd.DataFrame(logs)
logs_df.to_csv("logs_tactics_constraint.csv", index=False)

print("\nSaved:")
print("- retrieval_55_per_id_tactics_constraint.parquet", retrieval_df.shape)
print("- logs_tactics_constraint.csv", logs_df.shape)


[22] fallback=True | gold_id=T1566 gold_tac=Initial Access | pred_id=T1566 pred_id_tac=Initial Access | Predicted(Tactics)=Collection | PredTacMatch=NO | GoldTacMatch=OK | Check=PredTacticsMismatchButCorrect
[31] fallback=False | gold_id=T1047 gold_tac=Execution | pred_id=T1497 pred_id_tac=Defense Evasion/Discovery | Predicted(Tactics)=Discovery | PredTacMatch=OK | GoldTacMatch=NO | Check=OK
[32] fallback=False | gold_id=T1087 gold_tac=Discovery | pred_id=T1069 pred_id_tac=Discovery | Predicted(Tactics)=Discovery | PredTacMatch=OK | GoldTacMatch=OK | Check=OK
[48] fallback=False | gold_id=T1036 gold_tac=Defense Evasion | pred_id=T1036 pred_id_tac=Defense Evasion | Predicted(Tactics)=Defense Evasion | PredTacMatch=OK | GoldTacMatch=OK | Check=OK
[49] fallback=False | gold_id=T1036 gold_tac=Defense Evasion | pred_id=T1036 pred_id_tac=Defense Evasion | Predicted(Tactics)=Defense Evasion | PredTacMatch=OK | GoldTacMatch=OK | Check=OK
[124] fallback=False | gold_id=T1016 gold_tac=Discovery 

In [1]:
import pandas as pd

df_logs = pd.DataFrame(logs)

save_path = "MITSU_FINAL_SECOND_again.csv"
df_logs.to_csv(save_path, index=False, encoding="utf-8")

print("Saved:", save_path)
df_logs.head()


NameError: name 'logs' is not defined

In [ ]:
print(test.columns)



### Evaluate

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 正解ラベル（ATT&CK Technique ID）
y_true = test["act_Technique ID"].values

# 予測ラベル（Llama の出力）
# predicted_labels は Llama 推論ループで append していたリスト
y_pred = predicted_labels

# ---- 評価 ----
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"Accuracy:  {accuracy}")
print(f"Precision: {precision}")
print(f"Recall:    {recall}")
print(f"F1 Score:  {f1}")


In [ ]:
# import pandas as pd

# # ① CSV を読み込み
# df_base = pd.read_csv("/workspace/miluser/Mitsuharu_2002/csv_data/TTPHunter_dataset.csv")              # Domain, Technique ID, Technique Name, Sentences
# df_pred = pd.read_csv("/workspace/miluser/Mitsuharu_2002/predicted_Tactics_70B_1678.csv")     # Technique ID, Tactics Name, Sentences, Predicted

# # （もし念のためヘッダに余計な空白がありそうなら）
# df_base.columns = df_base.columns.str.strip()
# df_pred.columns = df_pred.columns.str.strip()

# # ② Predicted だけ使う（＋キーになる Sentences）
# df_pred_small = df_pred[["Sentences", "Predicted"]]

# # ③ Sentences をキーに結合（LEFT JOIN）
# df_merged = df_base.merge(df_pred_small, on="Sentences", how="left")

# # ④ 保存
# df_merged.to_csv("TTPHunter_with_Predicted.csv", index=False)


In [ ]:
["\nClassification:"+str(score)+"\n" for doc, score in search_results]

以下追加分（混合行列）

In [ ]:
import sys
print(sys.executable)

!pip install --upgrade pip


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred, labels=sorted(list(set(y_true))))
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=True, yticklabels=True)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

import pandas as pd

print(pd.Series(y_pred).value_counts())



In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# ラベルを揃えておく
labels = sorted(list(set(y_true)))

cm = confusion_matrix(y_true, y_pred, labels=labels)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,      # ★ 数字を表示
    fmt="d",         # ★ 整数表示（floatなら ".2f" とか）
    cmap="Blues",
    xticklabels=labels,
    yticklabels=labels
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

print(pd.Series(y_pred).value_counts())


In [ ]:
for t,p,txt in zip(y_true, y_pred, test["text"]):
    if t != p:
        print("True:", t)
        print("Pred:", p)
        print("Text:", txt)
        print("-" * 50)



### Few Shot learning 2

In [ ]:
# Aggregate predictions using majority voting
from collections import Counter

def majority_voting(predictions):
    count = Counter(predictions)
    return count.most_common(1)[0][0]

In [ ]:
import random

def create_prompt(template, search_results, text_to_predict):
    xs = ["\nText:"+ doc.page_content.lower()+"\nClassification:"+doc.metadata['TTP_ID'] for doc, _ in search_results]
    xs = ''.join(xs)+"\nText:"+ text_to_predict.lower() +"\nClassification:"
    return template + xs

def split_list(lst):
    random.shuffle(lst)  # Shuffle the list to ensure randomness
    n = len(lst)
    third = n // 3
    remainder = n % 3

    # Determine the splitting points
    split1 = third
    split2 = third * 2

    if remainder == 1:
        split1 += 1
    elif remainder == 2:
        split1 += 1
        split2 += 1

    # Split the list
    first_third = lst[:split1]
    second_third = lst[split1:split2]
    third_third = lst[split2:]

    return first_third, second_third, third_third



predicted_labels = []
# Iterating over the DataFrame row by row
for index, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    search_results = db.similarity_search_with_score(text_to_predict, k=150)

    template1 = "Your task is to classify the last sentence of 'Text' based on the provided examples. Each example shows a sentence and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n"
    template2 = "Please classify the last sentence of 'Text' based on the provided examples. Each example shows a sentence and its classification label. Use the patterns from these examples to classify the given text correctly. Return only the classification label starting with 'T'.\n"
    template3 = "Your task is to determine the classification of the last sentence of 'Text' using the provided examples. Each example pairs a sentence with its classification label. Analyze these examples to classify the given text. Output only the classification label starting with 'T'.\n"

    ## Split the list
    first_part, second_part, third_part = split_list(search_results)

    
    prompt1 = create_prompt(template1, first_part, text_to_predict)
    prompt2 = create_prompt(template2, second_part, text_to_predict)
    prompt3 = create_prompt(template3, third_part, text_to_predict)
    
    # Collect predictions
    prompts = [prompt1, prompt2, prompt3]
    predictions = []
    
    for prompt in prompts:
        
        prediction = llm.invoke(prompt).content
        predictions.append(prediction)

    print(predictions)
    
    
    final_prediction = majority_voting(predictions)
    #print("Final Prediction:", final_prediction)
    predicted_labels.append(final_prediction)
    print("actual:", row["Technique ID"], ", predicted:", final_prediction)

### Evaluate

In [ ]:
cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]

# Example vectors containing strings
y_true = test["Technique ID"].values#["cat", "dog", "fish", "cat", "dog"]
#y_true = train_data.Key
y_pred = cleaned_labels#.values#["cat", "dog", "cat", "cat", "dog"]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1}")

temperature:0, k=45

Accuracy: 0.9523809523809523
Precision: 0.9761904761904762
Recall: 0.9523809523809523
F1 Score: 0.9603174603174602

temperature:1, k=45

Accuracy: 0.9285714285714286
Precision: 0.9642857142857143
Recall: 0.9285714285714286
F1 Score: 0.9365079365079364

### V2 - SecureBERT Embeddings

In [ ]:
### embedding
model2 = "ehsanaghaei/SecureBERT"
embeddings2 = HuggingFaceEmbeddings(model_name = model2)

docs2 = []
for index, row in train.iterrows():
    doc = Document(page_content=row["Sentences"], metadata={"TTP_ID": row["Technique ID"]})
    docs2.append(doc)

print(len(docs2))

In [ ]:
db2 = FAISS.from_documents(docs2, embeddings2)


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings3 = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

db3 = FAISS.from_documents(docs2, embeddings3)


In [ ]:
db2

In [ ]:
from langchain_openai import ChatOpenAI

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)###, model_kwargs={"top_p": 0.0})
#llm = ChatOpenAI(model="o1-mini", temperature=1)
#llm = ChatOpenAI(model="chatgpt-4o-latest", temperature=0)

#llm.invoke(prompt)

scores_list = []
predicted_labels = []
# Iterating over the DataFrame row by row
for index, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    ##### Prompt 1
    search_results = db3.similarity_search_with_score(text_to_predict, k=45)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = "Your task is to classify the last sentence of 'InputText' based on the provided examples. Each example shows a sentence, inverse similarity to InputText (the lower the better), and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n" + xs
    #xs = "You are an expert in text classification. Below are examples of texts with their classifications based on their similarity to the 'InputText'. For the 'InputText' provided, choose the most appropriate classification based on the examples given. Only return the classification label (e.g., TXXXX). Use the closest similarity as the primary criterion for classification.\n\n" + xs
    #####
    [scores_list.append(score) for doc, score in search_results]
    ### search
    '''
    search_results = db.similarity_search_with_score(text_to_predict, k=55)
    xs = ["\n"+ "Example "+ str(i) +":\nText:"+ doc.page_content.lower()+"\nClassification:"+doc.metadata['TTP_ID']+"\n" for i, (doc, _) in enumerate(search_results)]
    ##xs = ["\nText:"+" ".join(str(doc.page_content).lower().split())+"\nClassification:"+doc.metadata['TTP_ID'] for doc, _ in search_results]
    

    ### mmr search
    #search_results = db.max_marginal_relevance_search(text_to_predict,k=45, fetch_k=250)
    #xs = ["\nText:"+doc.page_content.lower()+"\nClassification:"+doc.metadata['TTP_ID'] for doc in search_results]

    
    xs += "\n\nNote: The examples show a pattern where certain attack behaviors correspond to specific classification labels. Use this pattern recognition to classify the new text accurately.\nNow, classify the following text:\n"
    xs = ''.join(xs)+"\nText:"+ text_to_predict.lower() +"\nClassification:"
    #xs = "Your task is to classify the last sentence of 'Text' based on the provided examples. Each example shows a sentence and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n" + xs
    xs = "Your task is to classify the last sentence of 'Text' based on the diverse examples provided below. Learn the patterns from these examples to make accurate classifications. Output only the classification label beginning with 'T'.\n\nExamples:\n" + xs
    '''
    #print(xs)
    prompt = xs#prompt#"Write a poem about a cat chasing a butterfly."
    result = llm.invoke(prompt)
    predicted_label = result.content
    
    #predicted_label = most_common_ttp_id
    #print(predicted_label)
    predicted_labels.append(predicted_label)
    print("actual:", row["Technique ID"], ", predicted:", predicted_label)
    #break

### No LLM Call

In [ ]:
from langchain_openai import ChatOpenAI

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)###, model_kwargs={"top_p": 0.0})
#llm = ChatOpenAI(model="chatgpt-4o-latest", temperature=0)

#llm.invoke(prompt)

scores_list = []
predicted_labels = []
# Iterating over the DataFrame row by row
for index, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    ##### Prompt 1
    search_results = db2.similarity_search_with_score(text_to_predict, k=45)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = "Your task is to classify the last sentence of 'InputText' based on the provided examples. Each example shows a sentence, inverse similarity to InputText (the lower the better), and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n" + xs
    #xs = "You are an expert in text classification. Below are examples of texts with their classifications based on their similarity to the 'InputText'. For the 'InputText' provided, choose the most appropriate classification based on the examples given. Only return the classification label (e.g., TXXXX). Use the closest similarity as the primary criterion for classification.\n\n" + xs
    #####
    [scores_list.append(score) for doc, score in search_results]
    
    #print(xs)
    #prompt = xs#prompt#"Write a poem about a cat chasing a butterfly."
    #result = llm.invoke(prompt)
    #predicted_label = result.content
    
    #predicted_label = most_common_ttp_id
    #print(predicted_label)
    #predicted_labels.append(predicted_label)
    #print("actual:", row["Technique ID"], ", predicted:", predicted_label)
    #break

In [ ]:
### Evaluate

In [ ]:
cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]

# Example vectors containing strings
y_true = test["Technique ID"].values#["cat", "dog", "fish", "cat", "dog"]
#y_true = train_data.Key
y_pred = cleaned_labels#.values#["cat", "dog", "cat", "cat", "dog"]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1}")

In [ ]:
# Secure Bert Embeddings - 
Accuracy: 0.9523809523809523
Precision: 0.9880952380952381
Recall: 0.9523809523809523
F1 Score: 0.9619047619047618

In [ ]:
from statistics import mean
print(max(scores_list))
print(mean(scores_list))
print(min(scores_list))

In [ ]:
from statistics import mean
print(max(scores_list))
print(mean(scores_list))
print(min(scores_list))

In [ ]:
# Find incorrectly classified labels
incorrect_classifications = []
for index, (true_label, pred_label) in enumerate(zip(y_true, y_pred)):
    if true_label != pred_label:
        incorrect_classifications.append({
            'Index': index,
            'True Label': true_label,
            'Predicted Label': pred_label
        })

# Output the results
if incorrect_classifications:
    print("Incorrectly Classified Labels:")
    for item in incorrect_classifications:
        print(f"Index {item['Index']}: True Label = {item['True Label']}, Predicted Label = {item['Predicted Label']}")
else:
    print("All labels are correctly classified.")

In [ ]:
print(xs)

In [ ]:
train["Technique ID"].value_counts()["T1036"]#.head()

### SImilarity Investigation

In [ ]:
score_list = []
very_similar_string = "kerrdown can use a vbs base64 decoder function published by motobit"
totaly_wrong_string = "we have a nice weather today"
search_results2 = db2.similarity_search_with_score(very_similar_string, k=45)
[score_list.append(score) for doc, score in search_results2]
["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results2 if score < 2]
#score_list

### V3

In [ ]:
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)###, model_kwargs={"top_p": 0.0})
#llm = ChatOpenAI(model="chatgpt-4o-latest", temperature=0)

#llm.invoke(prompt)

def predict_label(text_to_predict):
    ##### Prompt 1
    search_results = db2.similarity_search_with_score(text_to_predict, k=45)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = """You are a MITRE ATT&CK TTP classification expert.
    Your task is to classify the 'InputText' based on the provided examples below.
    Each example shows a sentence, inverse similarity to InputText (the lower the better),
    and its corresponding classification label starting with 'T'.
    Use these examples to determine the correct classification for the given text (InputText).
    If 'InputText' is completely not related to any of the provided examples, return class 'T0000'.
    To determine if the 'InputText' is not related to the given examples, you can use the 'Similarity' propery of the examples.
    Low 'Similarity' values indicate the high similarity in strings.
    Return only the classification label starting with 'T' or 'T0000' if you found no appropriate class for 'InputText'.\n""" + xs
    #xs = "You are an expert in text classification. Below are examples of texts with their classifications based on their similarity to the 'InputText'. For the 'InputText' provided, choose the most appropriate classification based on the examples given. Only return the classification label (e.g., TXXXX). Use the closest similarity as the primary criterion for classification.\n\n" + xs
    #####
    
    #print(xs)
    prompt = xs#prompt#"Write a poem about a cat chasing a butterfly."
    result = llm.invoke(prompt)
    predicted_label = result.content
    return predicted_label


scores_list = []
predicted_labels = []
# Iterating over the DataFrame row by row
for index, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    predicted_label = predict_label(text_to_predict)
    
    #predicted_label = most_common_ttp_id
    #print(predicted_label)
    predicted_labels.append(predicted_label)
    print("actual:", row["Technique ID"], ", predicted:", predicted_label)
    #break

In [ ]:
print(xs)

In [ ]:
cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]

# Example vectors containing strings
y_true = test["Technique ID"].values#["cat", "dog", "fish", "cat", "dog"]
#y_true = train_data.Key
y_pred = cleaned_labels#.values#["cat", "dog", "cat", "cat", "dog"]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1}")

In [ ]:
predict_label("We have a beautiful wether today")

### Text to Predict

In [ ]:
text_to_predict = """
sofacy uses dealerschoice to target european government agency
robert falcone	by robert falcone
march 15, 2018 at 1:00 pm
category: unit 42 tags: dealerschoice, european government agency, sofacy summary back in october 2016, unit 42 published an initial analysis on a flash exploitation framework used by the sofacy threat group called dealerschoice. the attack consisted of microsoft word delivery documents that contained adobe flash objects capable of loading additional malicious flash objects embedded in the file or directly provided by a command and control server. sofacy continued to use dealerschoice throughout the fall of 2016, which we also documented in our december 2016 publication discussing sofacy s larger campaign. on march 12 and march 14, we observed the sofacy group carrying out an attack on a european government agency involving an updated variant of dealerschoice. the updated dealerschoice documents used a similar process to obtain a malicious flash object from a c2 server, but the inner mechanics of the flash object contained significant differences in comparison to the original samples we analyzed. one of the differences was a particularly clever evasion technique: to our knowledge this has never been observed in use. with the previous iterations of dealerschoice samples, the flash object would immediately load and begin malicious tasks. in the march attacks, the flash object is only loaded if the user scrolls through the entire content of the delivery document and views the specific page the flash object is embedded on. also, dealerschoice requires multiple interactions with an active c2 server to successfully exploit an end system. the overall process to result in a successful exploitation is: user must open the microsoft word email attachment
user must scroll to page three of the document, which will run the dealerschoice flash object
the flash object must contact an active c2 server to download an additional flash object containing exploit code
the initial flash object must contact the same c2 server to download a secondary payload
victim host must have a vulnerable version of flash installed the attack the attack involving this updated variant of dealerschoice was targeting a european government organization. the attack relied on a spear-phishing email with a subject of defence & security 2018 conference agenda that had an attachment with a filename of defence & security 2018 conference . the attached document contains a conference agenda that the sofacy group appears to have copied directly from the website for the underwater defence & security 2018 conference here. opening the attached defence & security 2018 conference file does not immediately run malicious code to exploit the system. instead, the user must scroll to the third page of the document, which will load a flash object that contains actionscript that will attempt to exploit the user s system to install a malicious payload. the flash object embedded within this delivery document is a variant of an exploit tool that we call dealerschoice. this suggests that the sofacy group is confident that the targeted individuals would be interested enough in the content to peruse through it. we analyzed the document to determine the reason that the malicious flash object only ran when the user scrolled to the third page. according to the file, the dealerschoice loader swf exists after the covert-shores- image file within the delivery document. this image file exists on the third page of the document, so the user would have to scroll down in the document to this third page to get the swf file to run. the user may not notice the flash object on the page, as word displays it as a tiny black box in the document, as seen in figure 1. this is an interesting anti-sandbox technique, as it requires human interaction prior to the document exhibiting any malicious activity. updated dealerschoice this dealerschoice flash object shares a similar process to previous variants; however, it appears that the sofacy actors have made slight changes to its internal code. also, it appears that the actors used actionscript from an open source video player called f4player , which is freely available on github with the following description: f4player is an open source flash (as3) video player and library project. it is so small that it is only 10kb (with skin file) and totally free under gpl license. the sofacy developer modified the f4player s actionscript to include additional code to load an embedded flash object. the additions include code to decrypt an embedded flash object and an event handler that calls a newly added function ( skinevent2 ) that plays the decrypted object, as seen in the code snippet below: the above code allows dealerschoice to load a second swf object, specifically loading it with an argument that includes a c2 url of hxxp://ndpmedia24[.]com/0pq6m4f.m3u8 . the embedded swf extracts the domain from the c2 url passed to it and uses it to craft a url to get the server s file in order to obtain permissions to load additional flash objects from the c2 domain. the actionscript relies on event listeners to call specific functions when the event event.complete is triggered after successful http requests are issued to the c2 server. the event handlers call functions with the following names, which includes an incrementing number that represents the order in which the functions are called: with these event handlers created, the actionscript starts by gathering system data from the flash.system.capabilities.serverstring property (just like in the original samples) and issues an http get with the system data as a parameter to the c2 url that was passed as an argument to the embedded swf when it was initially loaded. when this http request completes, the event listener will call the onload1 function. the onload1 function parses the response data from the request to the c2 url using regular expressions. according to the following code snippet, it appears the regular expression is looking for a hexadecimal string after / and before /sec , as well as any string between /hls/ and /tracks : the regular expressions suggest that the c2 server responds with content that is meant to resemble http live steaming (hls) traffic, which is a protocol that uses http to deliver audio and video files for streaming. the use of hls coincides with the use of actionscript code from the f4player to make the traffic seem legitimate. the variables storing the results of the regular expression matches are used within the actionscript for further interaction with the c2 server. the following is a list of these variables and their purpose: the onload1 function then sends an http get request to the c2 domain using the value stored in the r3 variable as a url. when this http request completes, the event listener will call the onload2 function. the onload2 function decrypts the response received from the http request issued in onload1 function. it does so by calling a sub-function to decrypt the content, using the value stored in the r1 variable as a key. the sub-function to decrypt the content skips the first 4 bytes, suggesting that the first four bytes of the downloaded content is in cleartext (most likely the fws or cws header to look legitimate). after decrypting the content, the onload2 function will issue another http get request with the system data as a parameter, but this time to the c2 using a url from the r4 variable. when this request completes, the event listener will call the onload3 function. the onload3 function will take the response to the http request in onload2 and treat it as the payload. the actionscript will read each byte of the c2 response and get the hexadecimal value of each byte and create a text array of 4-byte hexadecimal values with 0x prepended and , appended to each using the following code: this hexadecimal string will most likely be a string of shellcode that will contain and decrypt the ultimate portable executable (pe) payload. the string of comma separated hexadecimal values is passed as a parameter when loading the swf file downloaded in onload2 . this function creates an event listener for when the swf file is successfully loaded, which will call the onload5 function. the onload5 function is responsible for adding the newly loaded swf object as a child object to the current running object using the following code: this loads the swf file, effectively running the malicious code on the system. during our analysis, we were unable to coerce the c2 into providing a malicious swf or payload. as mentioned in our previous blogs on dealerschoice, the payload of choice for previous variants was sofacycarberp (seduploader), but we have no evidence to suggest this tool was used in this attack. we are actively researching and will update this blog in the event we discover the malicious flash object and payload delivered in this attack. linkage to prior campaign the delivery document used in this attack was last modified by a user named nick daemoji , which provides a linkage to previous sofacy related delivery documents. the previous documents that used this user name were macro-laden delivery documents that installed sofacycarberp/seduploader payloads, as discussed in talos blog. this overlap also points to a similar social engineering theme between these two campaigns, as both used content from upcoming military and defense conferences as a lure. conclusion the sofacy threat group continues to use their dealerschoice framework to exploit flash vulnerabilities in their attack campaigns. in the most recent variant, sofacy modified the internals of the malicious scripts, but continues to follow the same process used by previous variants by obtaining a malicious flash object and payload directly from the c2 server. unlike previous samples, this dealerschoice used a docx delivery document that required the user to scroll through the document to trigger the malicious flash object. the required user interaction turned out to be an interesting anti-sandbox technique that we had not seen this group perform in the past.
"""

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

text = "Hello world. How are you? I'm doing great!"
sentences = sent_tokenize(text)

print(sentences)

In [ ]:
###
text_sentences = sent_tokenize(text_to_predict)

text_ttps = []
for sentence in text_sentences:
    text_ttp = predict_label(sentence)
    print(sentence, " ->", text_ttp, "\n---")
    text_ttps.append(text_ttp)

In [ ]:
set(text_ttps)

In [ ]:
### 
predict_label("Process Hashes and Process Privilege Checks Upon being loaded, the malware starts a subroutine that hashes each running process on the system, and compares each hash with 3 hardcoded hashes: 0x6403527E → avp.exe associated with Kaspersky AV 0x23214B44 → ns.exe associated with Norton Security 0x651B3005 → ccSvcHst.exe associated with Symantec A global flag PROC_FLAG is used to track which of the 3 processes are running on the system. PROC_FLAG")

In [ ]:
predict_label("PROC_FLAG")

### V4 - DB Index Optimization

In [ ]:
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import KMeans

# Load SecureBERT
tokenizer = AutoTokenizer.from_pretrained(model2)
model = AutoModel.from_pretrained(model2)

# Move model to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate SecureBERT embeddings for a single text
def get_embedding(text):
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**tokens)
    # Pool the last hidden state to get sentence embedding
    embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Move embedding to CPU
    return embedding

# Extract texts and labels
texts = list(train["Sentences"].values)  # List of text items
labels = list(train["Technique ID"].values)  # List of labels

# Get SecureBERT embeddings for each document, item by item
embeddings = []

for text in texts:
    embedding = get_embedding(text)
    embeddings.append(embedding)

# Convert list of embeddings to a numpy array
embeddings = np.vstack(embeddings)  # Stack embeddings vertically to create a single array

# Normalize embeddings (L2 normalization)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)





In [ ]:
# Normalize embeddings (L2 normalization)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Supervised clustering using labels

# Get unique labels
unique_labels = set(labels)
n_classes = len(set(labels))
label_embeddings = {label: [] for label in unique_labels}

# Group embeddings by label
for embedding, label in zip(embeddings, labels):
    label_embeddings[label].append(embedding)

# Create label-guided centroids using KMeans for each class
centroids = []
for label, emb_list in label_embeddings.items():
    emb_array = np.array(emb_list)
    if len(emb_list) > 1:
        # Use KMeans to create centroids for each class, useful if there are multiple samples per class
        kmeans = KMeans(n_clusters=1, random_state=42)
        kmeans.fit(emb_array)
        centroids.append(kmeans.cluster_centers_[0])
    else:
        # If only one sample, use it directly as the centroid
        centroids.append(emb_array[0])

# Convert centroids to a numpy array
centroids = np.array(centroids)

# Initialize FAISS Index
d = embeddings.shape[1]  # dimension of the embeddings
nlist = n_classes  # number of clusters, one per class

#quantizer = faiss.IndexFlatL2(d)  # the quantizer used for clustering
#index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

index = faiss.IndexFlatL2(d)

# Train the FAISS index with the supervised centroids
#index.train(centroids)
#index.train(embeddings)
# Add embeddings to the index
index.add(embeddings)



In [ ]:
centroids

In [ ]:
# Example query


query_texts = ["Search the document with relevant content."]
query_embeddings = get_embedding(query_texts)
query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)

# Perform search
k = 45  # number of nearest neighbors
distances, indices = index.search(query_embeddings, k)

print(distances)

'''
# Print the search results
for i, query in enumerate(query_texts):
    print(f"Query: {query}")
    for j in range(k):
        if indices[i][j] != -1:  # valid index check
            print(f"Neighbor {j+1}: Document: {documents[indices[i][j]]['text']}, Distance: {distances[i][j]}")
'''

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)###, model_kwargs={"top_p": 0.0})
predicted_labels = []
scores_list = []
for i, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    query_texts = text_to_predict#["Search the document with relevant content."]
    query_embeddings = get_embedding(query_texts)
    query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
    
    # Perform search
    k = 60  # number of nearest neighbors
    distances, indices = index.search(query_embeddings, k)

    retrieved_docs = [texts[i] for i in indices[0] if i != -1]
    retrieved_labels = [labels[i] for i in indices[0] if i != -1]


    
    #[print(doc, score, label) for doc, score, label in zip(retrieved_docs, distances, labels) if score < 2]
    
    xs = ["\nText:"+ doc.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+label+"\n" for doc, score, label in zip(retrieved_docs, distances.flatten(), retrieved_labels) if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = "Your task is to classify the last sentence of 'InputText' based on the provided examples. Each example shows a sentence, inverse similarity to InputText (the lower the better), and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n" + xs
    
    #####
    
    #print(xs)
    prompt = xs#prompt#"Write a poem about a cat chasing a butterfly."
    result = llm.invoke(prompt)
    predicted_label = result.content

    predicted_labels.append(predicted_label)
    print("actual:", row["Technique ID"], ", predicted:", predicted_label)
    
    
    #print(distances)
    #print("----")
   
    [scores_list.append(score) for score in distances.flatten()]

In [ ]:
cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]

# Example vectors containing strings
y_true = test["Technique ID"].values#["cat", "dog", "fish", "cat", "dog"]
#y_true = train_data.Key
y_pred = cleaned_labels#.values#["cat", "dog", "cat", "cat", "dog"]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1}")

In [ ]:
#scores_list

In [ ]:
from statistics import mean
print(max(scores_list))
print(mean(scores_list))
print(min(scores_list))

In [ ]:
len(retrieved_docs)